In [1]:
import pandas as pd

In [4]:
# load hcp json file using json 
import json 
hcp_dict = json.load(open('../data/nap_fr.json'))




In [ ]:
hcp_dict_formated = {}
for page in hcp_dict :
    print(page)
    page_occs = hcp_dict[page]
    for occ in page_occs :
        hcp_dict_formated[occ] = page_occs[occ]


In [11]:
# load esco occupations 
esco_df = pd.read_csv('../data/occupations_fr.csv')
esco_df

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2166,directeur technique/directrice technique,régisseuse\nrégisseur\ndirectrice technique\nr...,NaN,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN
1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,opérateur de tréfilerie/opératrice de tréfilerie,opérateur de tréfilerie\nopératrice de tréfilerie,NaN,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,contrôleur qualité des instruments de précisio...,contrôleuse qualité des instruments de précisi...,NaN,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN
3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,technicien de la sécurité du trafic aérien/tec...,technicienne de la sécurité du trafic aérien\n...,NaN,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,responsable yield manager,responsable des revenus hôteliers\nyield manag...,NaN,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2937,Occupation,http://data.europa.eu/esco/occupation/ff656b3a...,2120,démographe,NaN,NaN,released,2016-07-05T16:21:57Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN
2938,Occupation,http://data.europa.eu/esco/occupation/ff8d4065...,9612,opérateur de tri/opératrice de tri,opérateur de tri\nopératrice de tri,NaN,released,2017-01-17T13:22:45Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
2939,Occupation,http://data.europa.eu/esco/occupation/ffa4dd5d...,5414,convoyeur de fonds/convoyeuse de fonds,transporteur de fonds\ntransporteuse de fonds\...,NaN,released,2016-12-22T10:57:58Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,NaN
2940,Occupation,http://data.europa.eu/esco/occupation/ffade2f4...,2422,adjoint administratif dans la fonction publiqu...,adjointe administrative dans la fonction publi...,NaN,released,2016-07-05T16:17:26Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,NaN


In [152]:
uri_to_label = esco_df[['conceptUri', 'preferredLabel']].values
# 2d list to dict 
uri_to_label_dict = {}
for uri, label in uri_to_label :
    uri_to_label_dict[uri] = label
    

In [105]:
import unidecode
import re
from nltk.corpus import stopwords
stop_words = stopwords.words('french')
def clean_text(text):
    text = text.lower()
    # remove french stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # french stemming
   

    # remove accents
    text = unidecode.unidecode(text)
    # remove / 
    text = text.replace('/', ' ')
    # remove , 
    text = text.replace(',', ' ')
    # remove special characters
    text = re.sub('[^a-zA-Z.\d\s]', '', text)
    # remove double spaces
    text = re.sub(' +', ' ', text)
  
    return text
 
def remove_sx_alt(text):
    return text.split('/')[0]




In [139]:
# prepare esco occupations
esco_corpus = esco_df['preferredLabel'].apply(remove_sx_alt).apply(clean_text).values
esco_corpus = list(esco_corpus)
esco_ids = esco_df['conceptUri'].values
esco_ids = list(esco_ids)

# prepare hcp occupations
hcp_ids =list( hcp_dict_formated.keys())
hcp_corpus = [clean_text(hcp_dict_formated[occ_id]) for occ_id in hcp_dict_formated]

# stats 
print('hcp_corpus length : ', len(hcp_corpus))
print('esco_corpus length : ', len(esco_corpus))


hcp_corpus length :  1110
esco_corpus length :  2942


In [143]:
# add alt labels to esco corpus 
esco_corpus_alt = []
esco_alt_ids = []
# get not nan alt labels

esco_alt_labels = esco_df[esco_df['altLabels'].notna()][['conceptUri','altLabels']].values
for (conceptUri , alt_list) in esco_alt_labels : 
   
    # split by new line 
    alt_list = alt_list.split('\n')
    for alt in alt_list :
        # remove sx
        alt = alt.split('/')[0]
        # clean text
        alt = clean_text(alt)
        if alt not in esco_corpus_alt :  # avoid duplicates (FI)
            esco_corpus_alt.append(alt)
            esco_alt_ids.append(conceptUri)


len(esco_corpus_alt),len(set(esco_corpus_alt))

(10172, 10172)

In [144]:
# prepare corpus
vetc_copus = esco_corpus + esco_corpus_alt
refs = esco_ids + esco_alt_ids


In [178]:
# tfidf char based 
# fit on esco  corpus
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(3,3), min_df=1, )

esco_tfid= vectorizer.fit_transform(vetc_copus)

features_names = vectorizer.get_feature_names_out()
# info 
#print('features_names length : ', len(features_names))
print('esco space : ', esco_tfid.shape)

esco space :  (13114, 2938)


In [181]:
import pprint
pd.options.display.max_colwidth = 100

def get_top_matches(hcp_occ_query, n=10):
    """
    tfid : tfidf matrix 
    tfid_ids : tfidf matrix ids 
    n : number of top matches 
    """
    # trasform query
    hcp_occ_query_tfid = vectorizer.transform([hcp_occ_query])
    # cosine similarity
    cosine_similarities = esco_tfid.dot(hcp_occ_query_tfid.T).toarray().ravel()
    # sort
    top_matches_indices = cosine_similarities.argsort()[::-1]
    # map torefs  
    top_matches = [(refs[i], cosine_similarities[i]) for i in top_matches_indices]
    # pandify 
    top_matches_df = pd.DataFrame(top_matches, columns=['conceptUri', 'cosine_similarity'])
    # drop zero scores 
    top_matches_df = top_matches_df[top_matches_df['cosine_similarity'] > 0]
    # group by refs
    #top_matches_df = top_matches_df.groupby('conceptUri').sum()

    # sort
    top_matches_df = top_matches_df.sort_values('cosine_similarity', ascending=False)
    # map refs to labels 
    top_matches_df['label'] = top_matches_df.conceptUri.map(lambda x: uri_to_label_dict[x])
    top_matches_df['id'] = top_matches_df.index
    # reset index
    top_matches_df = top_matches_df.reset_index(drop=True)
    # return top n
    return top_matches_df[['label','cosine_similarity']].head(n)
    
    

# test
key = '1413'
idx = hcp_ids.index(key)
hcp_occ =list(hcp_dict_formated.items())[idx]
jcp_occ_id , hcp_occ_query = hcp_occ


print(f'top_matches_ids for :\n {jcp_occ_id} :  {hcp_occ_query} ')
print('--'*30)
# reomve text btwn parenthesis
hcp_occ_query = re.sub('\(.*\)', '', hcp_occ_query)
hcp_occ_query = clean_text(hcp_occ_query)

top_matches_ids = get_top_matches(hcp_occ_query)
print(top_matches_ids)


top_matches_ids for :
 1413 :  Professeurs universitaires en sciences pures (biologie, géologie, physique, chimie, mathématiques, informatique..) 
------------------------------------------------------------
                                                                                                 label  \
0                                                          enseignant-chercheur/enseignante-chercheuse   
1                                                          enseignant-chercheur/enseignante-chercheuse   
2     professeur de sciences de la vie et de la Terre/professeure de sciences de la vie et de la Terre   
3  maître de conférences des universités-praticien hospitalier de médecine/maîtresse de conférences...   
4     professeur de sciences de la vie et de la Terre/professeure de sciences de la vie et de la Terre   
5     professeur de sciences de la vie et de la Terre/professeure de sciences de la vie et de la Terre   
6       professeur de sciences économiques et soci

In [96]:
# TODO
# add alt labels and agg-sum scores conceptUri
# control rules (atleast one token)


In [112]:
for occ in esco_corpus:
    print(occ)


directeur technique
operateur trefilerie
controleur qualite instruments precision
technicien securite trafic aerien
responsable yield manager
aidelaborantin danalyses medicales
technicien laboratoire asphaltes
assistant pedagogique ecole elementaire
physiotherapeute
professeur theatre
operateur machine hydrogenation
operateur machine production pates alimentaires
debitant tabac
responsable comptoir location dautres machines equipements biens materiels
geometre minier
responsable securite sanitaire agroalimentaire
technicien dengins agricoles
conducteur dautobus
responsable importexport viandes produits base viande
charge mission logement
chef cuisinier
conducteur machines produits non tisses
directeur detablissement denseignement superieur
superviseur poseurs isolation thermique
ingenieur demantelement deconstruction
charge dindemnisation assurances
directeur artistique habillement
operateur production nitroglycerine
ingenieur biotechnologies sante
tuteur
tisseur
carreleur
gerant magas